In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Input, models, optimizers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image
from tqdm import tqdm
import re
from keras.preprocessing.image import img_to_array

ModuleNotFoundError: No module named 'tensorflow'

In [10]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
gpus = tf.config.list_physical_devices('GPU')
print(gpus)
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

Num GPUs Available:  0
[]


In [13]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices(device_type="GPU")
print(gpus)
print(tf.config.list_physical_devices())
tf.config.experimental.set_visible_devices(devices=gpus[0], device_type="GPU")
tf.config.experimental.set_memory_growth(device=gpus[0], enable=True)

[]
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


IndexError: list index out of range